# TF-IDF with spaCy

Date: 2024/05/03

spaCyのTokenizerを使ったTF-IDF順位付け全文検索

SQLiteと組み合わせる時は、SQLiteのWHERE...LIKE...で検索対象を絞り、それをspaCyでtokenizeして検索する。

In [1]:
text1 = '夕方に勉強していたら、遠くで犬が吠えているのが聞こえた。たぶん、散歩中の他の犬を怖がっているのだろう。'
text2 = '毎日、通勤時間に勉強する。勉強は自己への投資。'
text3 = '勉強しない上司は嫌い。'
text4 = '駅まで歩いている途中、散歩中の犬が私に向かって吠えた。'

texts = [text1, text2, text3, text4]

keywords = ' 勉強,　犬'

In [2]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load('ja_core_news_sm')

In [3]:
patterns = [[{"TEXT": e.strip()}] for e in keywords.split(',')]
patterns

[[{'TEXT': '勉強'}], [{'TEXT': '犬'}]]

In [4]:
matcher = Matcher(nlp.vocab)
matcher.add("日常", patterns)

In [5]:
for doc in nlp.pipe(texts):
    print(f'[len {len(doc)}]', end=' ')
    for token in doc:
        print(token.text, end=' ')
    print()

[len 36] 夕方 に 勉強 し て い たら 、 遠く で 犬 が 吠え て いる の が 聞こえ た 。 たぶん 、 散歩 中 の 他 の 犬 を 怖 がっ て いる の だろう 。 
[len 15] 毎日 、 通勤 時間 に 勉強 する 。 勉強 は 自己 へ の 投資 。 
[len 7] 勉強 し ない 上司 は 嫌い 。 
[len 19] 駅 まで 歩い て いる 途中 、 散歩 中 の 犬 が 私 に 向かっ て 吠え た 。 


## TF

In [6]:
all_tf = []
all_span = []

for doc in nlp.pipe(texts):
    cnt = {}
    span_dict = {}
    print(f'[len {len(doc)}]', end=' ')
    matches = matcher(doc, as_spans=True)
    for span in matches:
        if span.text not in cnt:
            cnt[span.text] = 0
        cnt[span.text] += 1
        if span.text not in span_dict:
            span_dict[span.text] = []
        span_dict[span.text].append([span.start_char, span.end_char])
        
    tf = {}
    for k,v in cnt.items():
        tf[k] = v/len(doc)
    all_tf.append(tf)
    all_span.append(span_dict)

[len 36] [len 15] [len 7] [len 19] 

In [7]:
all_tf

[{'勉強': 0.027777777777777776, '犬': 0.05555555555555555},
 {'勉強': 0.13333333333333333},
 {'勉強': 0.14285714285714285},
 {'犬': 0.05263157894736842}]

In [8]:
all_span

[{'勉強': [[3, 5]], '犬': [[14, 15], [38, 39]]},
 {'勉強': [[8, 10], [13, 15]]},
 {'勉強': [[0, 2]]},
 {'犬': [[15, 16]]}]

## IDF

In [9]:
sent_cnt = {}
for tf in all_tf:
    for k,v in tf.items():
        if k not in sent_cnt:
            sent_cnt[k] = 0
        if v > 0:
            sent_cnt[k] += 1

sent_cnt

{'勉強': 3, '犬': 2}

In [10]:
import math

all_idf = {}
           
for k,v in sent_cnt.items():
    all_idf[k] = math.log(len(texts)/v)

all_idf

{'勉強': 0.28768207245178085, '犬': 0.6931471805599453}

## TF-IDF

In [11]:
all_tf_idf = []

for tf in all_tf:
    tf_idf = {}
    for k,v in tf.items():
        tf_idf[k] = v * all_idf[k]
    all_tf_idf.append(tf_idf)

all_tf_idf

[{'勉強': 0.007991168679216135, '犬': 0.03850817669777474},
 {'勉強': 0.03835760966023745},
 {'勉強': 0.04109743892168297},
 {'犬': 0.03648143055578659}]

## Sort

In [12]:
score = []
idx = 0
for e in all_tf_idf:
    s = []
    for k,v in e.items():
        s.append(v)
    score.append([max(s), idx, all_span[idx], texts[idx]])
    idx += 1

score.sort(reverse=True)
score

[[0.04109743892168297, 2, {'勉強': [[0, 2]]}, '勉強しない上司は嫌い。'],
 [0.03850817669777474,
  0,
  {'勉強': [[3, 5]], '犬': [[14, 15], [38, 39]]},
  '夕方に勉強していたら、遠くで犬が吠えているのが聞こえた。たぶん、散歩中の他の犬を怖がっているのだろう。'],
 [0.03835760966023745,
  1,
  {'勉強': [[8, 10], [13, 15]]},
  '毎日、通勤時間に勉強する。勉強は自己への投資。'],
 [0.03648143055578659, 3, {'犬': [[15, 16]]}, '駅まで歩いている途中、散歩中の犬が私に向かって吠えた。']]